In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import kaggle_evaluation.default_inference_server

RND = 42

# ============================================================
# 1. LOAD TRAIN DATA
# ============================================================
train = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv")

feature_cols = [
    'market_forward_excess_returns',
    'risk_free_rate'
]

# Drop NA for safety
train = train.dropna(subset=feature_cols + ['forward_returns'])

X = train[feature_cols]
y = train['forward_returns']

# ============================================================
# 2. TRAIN MODEL
# ============================================================
model = RandomForestRegressor(
    n_estimators=200,
    max_depth=10,
    min_samples_leaf=3,
    random_state=RND,
    n_jobs=-1
)

print("Training model...")
model.fit(X, y)
print("Training complete.")

# ============================================================
# 3. PREDICT FUNCTION
# ============================================================
def predict(test_row):
    """
    test_row: dict passed by the Kaggle evaluation server
    MUST return a single float
    """

    df = pd.DataFrame([test_row])

    # Ensure missing columns exist (fallback = 0)
    for col in feature_cols:
        if col not in df.columns:
            df[col] = 0.0

    # Select features in correct order
    X_test = df[feature_cols]

    # Predict returns an array → take first element
    pred = model.predict(X_test)[0]

    return float(pred)

# ============================================================
# 4. START INFERENCE SERVER
# ============================================================
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(
    predict
)

if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        ("/kaggle/input/hull-tactical-market-prediction/",)
    )

Training model...
Training complete.
